In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import sklearn
import keras
import sys
    
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from sklearn import model_selection
from keras.layers import Dense
from keras import regularizers
from keras import optimizers
from sklearn.svm import SVC

2023-11-05 16:33:49.404020: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 16:33:49.439651: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 16:33:49.440126: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 16:33:50.223544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Dividindo X e Y para o modelo

In [2]:
data = pd.read_csv('dados.csv', sep=',')
X = np.array(data.drop(['MADE'], axis=1))
y = np.array(data['MADE'])

## SVM

In [3]:
from sklearn.model_selection import GridSearchCV

# Usando cerca de 5% de todos os dados
X = X[:10000]
y = y[:10000]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=23, test_size=0.2)


param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)

grid.best_params_

grid_predictions = grid.predict(X_test)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.521 total time=   1.8s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.521 total time=   1.7s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.521 total time=   1.7s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.521 total time=   1.8s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.521 total time=   1.7s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.521 total time=   1.9s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.521 total time=   2.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.521 total time=   1.9s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.521 total time=   2.2s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.521 total time=   2.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.521 total time=   2.3s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [8]:
print('Ein: %0.4f' % (1 - accuracy_score(y_train, grid.predict(X_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, grid.predict(X_test))))

classification_report(y_test, grid.predict(X_test))

Ein: 0.3952
Eout: 0.3935


'              precision    recall  f1-score   support\n\n           0       0.59      0.67      0.63       987\n           1       0.63      0.54      0.58      1013\n\n    accuracy                           0.61      2000\n   macro avg       0.61      0.61      0.61      2000\nweighted avg       0.61      0.61      0.60      2000\n'

### Abaixo temos, com base nos testes do gridsearch, os melhores parâmetros C e gamma para o modelo

In [9]:
grid.best_params_

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

### Recriando o modelo com base nos melhores parâmetros encontrados no grid

In [10]:
# Dividindo o dataset em 2
kfold = KFold(n_splits=2, shuffle=True, random_state=0)

model = SVC(C=1, gamma=0.0001, kernel='rbf')

# Cross validation
results = cross_val_score(model, X_train, y_train, cv=kfold)
results.mean()

0.5860000000000001